# 智能体框架

## 基本概念

### 同步调用和流输出

illufly 默认采用流输出的方式，无论在单个智能体中，还是嵌套的多智能体结构中。<br>
配合 log 函数就可以实现流打印。

In [1]:
from illufly.chat import FakeLLM
from illufly.io import log

log(FakeLLM(), "hi")

这是一个模拟调用!



'这是一个模拟调用!'

### 异步调用和流输出

In [2]:
from illufly.chat import FakeLLM
from illufly.io import alog

await alog(FakeLLM(), "hi", sleep=1)

这是一个模拟调用!



'这是一个模拟调用!'

观察线程池情况：

In [3]:
FakeLLM.monitor_executors()

{'FAKE_LLM': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0}}

### 绑定

使用绑定变量实现跨智能体变量传递。<br>
这是 Runnable 子类的实例对象多有的统一规格。<br>

obj.desk 是一个只读属性，可以访问绑定变量的字典，字典中的键值主要包括：

| 变量       | 生命周期       | 详细说明 |
|:----------|:-------------:|:---------------------------------------------------------|
| knowledge | 可手工维护    | 用于检索增强，添加方法 add_knowledge(knowledge: List[str])    |
| data      | 可手工维护    | 用于数据分析，添加执行方法 add_data(data: pandas.DataFrame)    |
| task      | 运行时修改    | 提问或输入，大模型调用开始自动修改                |
| output    | 运行时修改    | 结果或输出，大模型调用结束自动修改                |
| draft     | 运行时修改    | 写作任务中已完成的草稿，例如扩写任务中自动修改      |
| outline   | 运行时修改    | 扩写提纲，在扩写任务中自动生成                   |
| state     | 定制时使用    | 以上不够用时，建议使用state字典来定制状态数据      |


### 从绑定读取数据到输入

### 在初始化时使用Template

**如果列表中是 Template 实例，就判定为 role 为 system 类型：**

In [1]:
from illufly.chat import ChatQwen
from illufly.types import Template

In [2]:
a = ChatQwen(memory=[Template("IDEA"), "请你开始"])
a("给我讲个笑话")
a.memory

new_chat
self.task 给我讲个笑话
当然，这里有一个笑话供您欣赏：

为什么袜子总是只丢一只？因为丢两只根本就不会发现。



[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n给我讲个笑话\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant',
  'content': '当然，这里有一个笑话供您欣赏：\n\n为什么袜子总是只丢一只？因为丢两只根本就不会发现。'}]

In [3]:
a.exported_vars

{'last_input': None,
 'last_output': '当然，这里有一个笑话供您欣赏：\n\n为什么袜子总是只丢一只？因为丢两只根本就不会发现。',
 'task': '给我讲个笑话'}

In [4]:
a.bound_vars

{'task'}

In [3]:
# 上面的是语法糖，这里是被转换后的形式
b = ChatQwen(
    memory=[
        {"role": "system", "content": Template("IDEA")},
        {"role": "user", "content": "请你开始"}
    ]
)
b.memory

[]

**如果提示语模板中包含的变量与绑定中的名称不同，可以在初始化时指定映射规则：**

In [4]:
ChatQwen(
    # task 是模板中的变量名，prompt 是宿主绑定中的变量名
    memory=[Template("IDEA", input_mapping={"task": "prompt"})]
)

**对于消息列表中的Template，在调用时从绑定动态填充数据：**

In [5]:
ChatQwen(
    memory=[Template("IDEA")]
)

## 智能体：直调大模型

### 对话

In [1]:
from illufly.chat import ChatQwen

qwen = ChatQwen()
qwen("你能帮我写一首关于兔子做梦的四句儿歌?")
qwen.memory

小兔子，梦中跳，  
胡萝卜，满天空飘。  
月亮船，载它游，  
梦里笑，乐悠悠。



[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '小兔子，梦中跳，  \n胡萝卜，满天空飘。  \n月亮船，载它游，  \n梦里笑，乐悠悠。'}]

### 使用系统提示语

In [2]:
from illufly.chat import ChatQwen

qwen = ChatQwen(memory=["你是一个专门写儿歌的作家，请根据我的提示写作。"])
qwen("来一首关于兔子的，四句")
qwen.memory

小白兔，蹦蹦跳，  
耳朵长，尾巴小。  
爱吃萝卜和青菜，  
快乐生活在林梢。



[{'role': 'system', 'content': '你是一个专门写儿歌的作家，请根据我的提示写作。'},
 {'role': 'user', 'content': '来一首关于兔子的，四句'},
 {'role': 'assistant',
  'content': '小白兔，蹦蹦跳，  \n耳朵长，尾巴小。  \n爱吃萝卜和青菜，  \n快乐生活在林梢。'}]

In [3]:
qwen.memory

[{'role': 'system', 'content': '你是一个专门写儿歌的作家，请根据我的提示写作。'},
 {'role': 'user', 'content': '来一首关于兔子的，四句'},
 {'role': 'assistant',
  'content': '小白兔，蹦蹦跳，  \n耳朵长，尾巴小。  \n爱吃萝卜和青菜，  \n快乐生活在林梢。'}]

In [4]:
from illufly.chat import ChatQwen
from illufly.types import Template

qwen = ChatQwen(memory=[Template(text="你是一个专门写儿歌的作家，请根据我的提示写作。")])
qwen("来一首关于兔子的，四句")
qwen.memory

小兔子白又白，耳朵长长蹦蹦跳。
爱吃萝卜和青菜，月光下蹦跶真快乐。



[{'role': 'system', 'content': '你是一个专门写儿歌的作家，请根据我的提示写作。'},
 {'role': 'user', 'content': '来一首关于兔子的，四句'},
 {'role': 'assistant', 'content': '小兔子白又白，耳朵长长蹦蹦跳。\n爱吃萝卜和青菜，月光下蹦跶真快乐。'}]

In [6]:
from illufly.chat import ChatQwen
from illufly.types import Template

qwen = ChatQwen(memory=[
    Template(
        text="你是一个专门写儿歌的作家，请根据我的提示写作。我的要求是：{{task}}"
    ),
    "要简练"
])
qwen("来一首关于兔子的，四句")
qwen.memory

小白兔，蹦又跳，  
长耳朵，摇啊摇。  
爱吃萝卜和青菜，  
快乐生活在林梢。



[{'role': 'system', 'content': '你是一个专门写儿歌的作家，请根据我的提示写作。我的要求是：来一首关于兔子的，四句'},
 {'role': 'user', 'content': '要简练'},
 {'role': 'assistant',
  'content': '小白兔，蹦又跳，  \n长耳朵，摇啊摇。  \n爱吃萝卜和青菜，  \n快乐生活在林梢。'}]

In [7]:
qwen.exported_vars

{'last_input': None,
 'last_output': '小白兔，蹦又跳，  \n长耳朵，摇啊摇。  \n爱吃萝卜和青菜，  \n快乐生活在林梢。',
 'task': '来一首关于兔子的，四句'}

In [9]:
qwen("换成两条小鱼")
qwen.memory

两条小鱼，水中游，  
摇摇尾巴，点点头。  
相互追逐多欢喜，  
波光里笑影悠悠。



[{'role': 'system', 'content': '你是一个专门写儿歌的作家，请根据我的提示写作。我的要求是：来一首关于兔子的，四句'},
 {'role': 'user', 'content': '要简练'},
 {'role': 'assistant',
  'content': '小白兔，蹦又跳，  \n长耳朵，摇啊摇。  \n爱吃萝卜和青菜，  \n快乐生活在林梢。'},
 {'role': 'user', 'content': '换成两条小鱼'},
 {'role': 'assistant',
  'content': '两条小鱼，水中游，  \n摇摇尾巴，点点头。  \n相互追逐多欢喜，  \n波光里笑影悠悠。'}]

## 智能体：使用工具

工具应当是一个智能体，而使用工具的过程实际上是 **「委托-回复」** 的过程。<br>
实际上，这也是多智能体运行的一种策略。

In [1]:
# 从函数定义工具
from illufly.chat import ChatQwen
from illufly.types import ToolAgent
from illufly.io import TextBlock, log

def get_current_weather(location: str):
    """获取城市的天气情况"""
    yield TextBlock("chunk", f"{location}今天是晴天。 ")

t = ToolAgent(get_current_weather)
log(t, "广州天气如何啊？")

广州天气如何啊？今天是晴天。 



In [14]:
q = ChatQwen(tools=[ToolAgent(get_current_weather)])
q.tools

[<Tool get_current_weather: 获取城市的天气情况>]

### 返回工具提示

In [2]:
# 在类定义中声明工具
q = ChatQwen(tools=[ToolAgent(get_current_weather)], exec_tool=False)
log(q, "今天广州天气如何啊", verbose=True)
q.memory

  1s [TOOLS_CALL_CHUNK] {"index": 0, "id": "call_8909e53a21914771b52ab5", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
  1s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
  1s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
  1s [TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
  1s [TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_8909e53a21914771b52ab5", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天是晴天。 广州今天是晴天。不过请注意，实际天气可能会有所变化，外出时最好还是确认一下实时天气。



[{'role': 'user', 'content': '今天广州天气如何啊'},
 {'role': 'assistant', 'content': ''},
 {'role': 'tool', 'content': '广州今天是晴天。 '},
 {'role': 'assistant',
  'content': '广州今天是晴天。不过请注意，实际天气可能会有所变化，外出时最好还是确认一下实时天气。'}]

In [6]:
q.toolkits

[<Tool get_current_weather: 获取城市的天气情况>]

**如果仅在调用时提供tools参数，则意味着并不希望执行工具，即使设置了exec_tool也是如此**

In [7]:
q.get_tools_desc([Tool(get_current_weather)])

[{'type': 'function',
  'function': {'name': 'get_current_weather',
   'description': '获取城市的天气情况',
   'parameters': {'type': 'object',
    'properties': {'location': {'type': 'str', 'description': ''}},
    'required': ['location']}}},
 {'type': 'function',
  'function': {'name': 'get_current_weather',
   'description': '获取城市的天气情况',
   'parameters': {'type': 'object',
    'properties': {'location': {'type': 'str', 'description': ''}},
    'required': ['location']}}}]

### 执行工具回调

In [8]:
# 在调用中声明工具
q = ChatQwen()
log(q, "今天广州天气如何啊", tools=[Tool(get_current_weather)], verbose=True)
q.memory

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_167e36b3be884ba885ec2a", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_167e36b3be884ba885ec2a", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天是晴天。 广州今天的天气是晴朗的。

[{'role': 'user', 'content': '今天广州天气如何啊'},
 {'role': 'assistant', 'content': ''},
 {'role': 'tool', 'content': '广州今天是晴天。 '},
 {'role': 'assistant', 'content': '广州今天的天气是晴朗的。'}]

In [2]:
# 在类定义中声明工具
q = ChatQwen(tools=[Tool(get_current_weather)], exec_tool=True)
log(q, "今天广州可以晒被子吗", verbose=True)
q.memory

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_f4c120cb63fa43d09fa1f5", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_f4c120cb63fa43d09fa1f5", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天是晴天。 今天广州是晴天，非常适合晒被子。

[{'role': 'user', 'content': '今天广州可以晒被子吗'},
 {'role': 'assistant', 'content': ''},
 {'role': 'tool', 'content': '广州今天是晴天。 '},
 {'role': 'assistant', 'content': '今天广州是晴天，非常适合晒被子。'}]

## 智能体团队：执行管道

### 从聊天开始的管道 [Chat, Chat, Chat]

In [1]:
from illufly.agent import ChatQwen, Pipe
from illufly.io import log

pipe = Pipe(
    ChatQwen(memory="我是一个儿童作家，擅长写儿歌。"),
    ChatQwen(memory="请你帮我评价文章特色，两句话即可"),
    ChatQwen(memory="请针对我的写作成果打一个分数，给出一句话的打分点，最终给出1分至5分")
)

log(pipe, "你能帮我写一首关于兔子做梦的？四句即可。")

[AGENT] >>> Node 1: ChatQwen
小白兔，梦中跳，月亮船上摇啊摇。
胡萝卜，满天星，梦里世界甜如蜜。

[AGENT] >>> Node 2: ChatQwen
这篇文章特色在于其富有童趣的想象力，通过生动活泼的语言营造了一个甜美梦幻的童话氛围，让人感受到纯真的快乐与奇妙的想象。

[AGENT] >>> Node 3: ChatQwen
4分。打分点在于文章成功运用了富有童趣的想象力和生动活泼的语言，创造出独特的童话氛围，给人以愉悦的阅读体验，但可能在深度或结构上有改进空间。



'4分。打分点在于文章成功运用了富有童趣的想象力和生动活泼的语言，创造出独特的童话氛围，给人以愉悦的阅读体验，但可能在深度或结构上有改进空间。'

In [2]:
pipe.runnables[0].memory

[{'role': 'system', 'content': '我是一个儿童作家，擅长写儿歌。'},
 {'role': 'user', 'content': '你能帮我写一首关于兔子做梦的？四句即可。'},
 {'role': 'assistant', 'content': '小白兔，梦中跳，月亮船上摇啊摇。\n梦见胡萝卜大又甜，醒来嘴角笑弯弯。'}]

### 从提示语构造开始的管道 [Template, Chat, Chat]

In [3]:
from illufly.agent import ChatQwen, Pipe, Template
from illufly.io import log, alog
from illufly.hub import Template

In [4]:
pipe = Pipe(
    ChatQwen(memory=[Template("IDEA", desk_map={"task": "task"}), "请你开始"]),
    ChatQwen(memory="请帮我对作品进行评价，找出三个缺点，一句话总结。")
)

log(pipe, "请你帮我写一首4行儿歌")

[AGENT] >>> Node 1: ChatQwen
小鸭子嘎嘎叫，  
摇摇摆摆水里漂。  
太阳笑眯眯，  
快乐时光乐逍遥。

[AGENT] >>> Node 2: ChatQwen
作品优点明显，充满童趣与自然和谐之美，但要指出三点可以改进的地方：

1. **缺乏情节展开**：诗歌描述了小鸭子和太阳的快乐场景，但没有一个具体的故事进展或冲突解决，可能让寻求深层含义的读者感到略显单薄。
2. **角色单一**：仅出现了小鸭子和太阳两个角色，引入更多动物或自然元素可以增加诗歌的丰富性和互动性。
3. **语言简单**：虽然符合儿童诗的风格，但对于提升文学价值和吸引更广泛年龄段的读者来说，适度增加词汇多样性和诗意表达会更有益。

一句话总结：这是一首洋溢着纯真快乐的儿童诗，若能在保持童趣的同时融入更多元的情节、角色和语言艺术，将更能吸引并打动不同层次的读者。



'作品优点明显，充满童趣与自然和谐之美，但要指出三点可以改进的地方：\n\n1. **缺乏情节展开**：诗歌描述了小鸭子和太阳的快乐场景，但没有一个具体的故事进展或冲突解决，可能让寻求深层含义的读者感到略显单薄。\n2. **角色单一**：仅出现了小鸭子和太阳两个角色，引入更多动物或自然元素可以增加诗歌的丰富性和互动性。\n3. **语言简单**：虽然符合儿童诗的风格，但对于提升文学价值和吸引更广泛年龄段的读者来说，适度增加词汇多样性和诗意表达会更有益。\n\n一句话总结：这是一首洋溢着纯真快乐的儿童诗，若能在保持童趣的同时融入更多元的情节、角色和语言艺术，将更能吸引并打动不同层次的读者。'

In [5]:
pipe.memory

[{'role': 'user', 'content': '节点 <0> 正在处理任务...'},
 {'role': 'assistant', 'content': '小鸭子嘎嘎叫，  \n摇摇摆摆水里漂。  \n太阳笑眯眯，  \n快乐时光乐逍遥。'},
 {'role': 'user', 'content': '节点 <1> 正在处理任务...'},
 {'role': 'assistant',
  'content': '作品优点明显，充满童趣与自然和谐之美，但要指出三点可以改进的地方：\n\n1. **缺乏情节展开**：诗歌描述了小鸭子和太阳的快乐场景，但没有一个具体的故事进展或冲突解决，可能让寻求深层含义的读者感到略显单薄。\n2. **角色单一**：仅出现了小鸭子和太阳两个角色，引入更多动物或自然元素可以增加诗歌的丰富性和互动性。\n3. **语言简单**：虽然符合儿童诗的风格，但对于提升文学价值和吸引更广泛年龄段的读者来说，适度增加词汇多样性和诗意表达会更有益。\n\n一句话总结：这是一首洋溢着纯真快乐的儿童诗，若能在保持童趣的同时融入更多元的情节、角色和语言艺术，将更能吸引并打动不同层次的读者。'}]

In [6]:
pipe.runnables[0].memory

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请你帮我写一首4行儿歌\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant', 'content': '小鸭子嘎嘎叫，  \n摇摇摆摆水里漂。  \n太阳笑眯眯，  \n快乐时光乐逍遥。'}]

In [9]:
pipe.runnables[1].memory

[{'role': 'system', 'content': '请帮我对作品进行评价，找出三个缺点，一句话总结。'},
 {'role': 'user', 'content': '小鸭子嘎嘎叫，  \n摇摇摆摆水里漂。  \n太阳笑眯眯，  \n快乐时光乐逍遥。'},
 {'role': 'assistant',
  'content': '作品优点明显，充满童趣与自然和谐之美，但要指出三点可以改进的地方：\n\n1. **缺乏情节展开**：诗歌描述了小鸭子和太阳的快乐场景，但没有一个具体的故事进展或冲突解决，可能让寻求深层含义的读者感到略显单薄。\n2. **角色单一**：仅出现了小鸭子和太阳两个角色，引入更多动物或自然元素可以增加诗歌的丰富性和互动性。\n3. **语言简单**：虽然符合儿童诗的风格，但对于提升文学价值和吸引更广泛年龄段的读者来说，适度增加词汇多样性和诗意表达会更有益。\n\n一句话总结：这是一首洋溢着纯真快乐的儿童诗，若能在保持童趣的同时融入更多元的情节、角色和语言艺术，将更能吸引并打动不同层次的读者。'},
 {'role': 'user', 'content': '找出三个优点吧'},
 {'role': 'assistant',
  'content': '当然，这首作品同样展现出了多个亮点：\n\n1. **生动的意象**：通过“小鸭子嘎嘎叫，摇摇摆摆水里漂”这样的生动描绘，读者可以轻易地在心中勾勒出一幅活泼有趣的画面，增强了诗歌的表现力。\n2. **积极的情感传递**：“太阳笑眯眯，快乐时光乐逍遥”营造了一种温暖、愉快的氛围，能够有效激发读者的正面情绪，传递了简单的快乐和对自然美的欣赏。\n3. **易于理解与共鸣**：简洁明了的语言和贴近生活的主题使得这首诗不仅适合儿童阅读，也能让成人找回童年的纯真感，易于引起广泛共鸣。\n\n一句话总结：这首诗歌以它简洁生动的描绘、积极向上的氛围及广泛的共鸣度，成功地打造了一个既愉悦又富有感染力的阅读体验。'}]

In [8]:
log(pipe.runnables[1], "找出三个优点吧")

当然，这首作品同样展现出了多个亮点：

1. **生动的意象**：通过“小鸭子嘎嘎叫，摇摇摆摆水里漂”这样的生动描绘，读者可以轻易地在心中勾勒出一幅活泼有趣的画面，增强了诗歌的表现力。
2. **积极的情感传递**：“太阳笑眯眯，快乐时光乐逍遥”营造了一种温暖、愉快的氛围，能够有效激发读者的正面情绪，传递了简单的快乐和对自然美的欣赏。
3. **易于理解与共鸣**：简洁明了的语言和贴近生活的主题使得这首诗不仅适合儿童阅读，也能让成人找回童年的纯真感，易于引起广泛共鸣。

一句话总结：这首诗歌以它简洁生动的描绘、积极向上的氛围及广泛的共鸣度，成功地打造了一个既愉悦又富有感染力的阅读体验。



'当然，这首作品同样展现出了多个亮点：\n\n1. **生动的意象**：通过“小鸭子嘎嘎叫，摇摇摆摆水里漂”这样的生动描绘，读者可以轻易地在心中勾勒出一幅活泼有趣的画面，增强了诗歌的表现力。\n2. **积极的情感传递**：“太阳笑眯眯，快乐时光乐逍遥”营造了一种温暖、愉快的氛围，能够有效激发读者的正面情绪，传递了简单的快乐和对自然美的欣赏。\n3. **易于理解与共鸣**：简洁明了的语言和贴近生活的主题使得这首诗不仅适合儿童阅读，也能让成人找回童年的纯真感，易于引起广泛共鸣。\n\n一句话总结：这首诗歌以它简洁生动的描绘、积极向上的氛围及广泛的共鸣度，成功地打造了一个既愉悦又富有感染力的阅读体验。'

## 智能体团队：从提纲扩写

### 构建提纲

In [15]:
from illufly.agent import ChatQwen, Pipe, FromOutline, Template
from illufly.io import log, alog

outline = Pipe(
    Template("OUTLINE"),
    ChatQwen()
)

In [16]:
log(outline, {"task": "写一首两段儿歌，每段20个字即可，策划简单一点"})

[AGENT] >>> Node 1: Template
[AGENT] >>> Node 2: ChatQwen
# 儿歌：小星星的梦

## 第一段：星空奇妙夜
<OUTLINE>
- 描述夜晚星空的美丽景象，提及小星星
- 引入小星星梦想飞翔的念头，充满好奇与期待
- 字数：约20字
</OUTLINE>

## 第二段：晨光中的舞者
<OUTLINE>
- 描述随着黎明到来，小星星与月亮道别的温馨场景
- 小星星化作流星，短暂而灿烂地“舞动”于天际
- 字数：约20字
</OUTLINE>



'# 儿歌：小星星的梦\n\n## 第一段：星空奇妙夜\n<OUTLINE>\n- 描述夜晚星空的美丽景象，提及小星星\n- 引入小星星梦想飞翔的念头，充满好奇与期待\n- 字数：约20字\n</OUTLINE>\n\n## 第二段：晨光中的舞者\n<OUTLINE>\n- 描述随着黎明到来，小星星与月亮道别的温馨场景\n- 小星星化作流星，短暂而灿烂地“舞动”于天际\n- 字数：约20字\n</OUTLINE>'

### 提纲 + 扩写

In [9]:
from illufly.agent import ChatQwen, Pipe, FromOutline
from illufly.io import log, alog
from illufly.hub import Template

writer = Pipe(
    ChatQwen(memory=[Template("OUTLINE"), "请开始编写提纲"]),
    FromOutline(ChatQwen(memory=[Template("FROM_OUTLINE")]))
)

log(writer, "写一首两段儿歌，每段20个字即可，策划简单一点")

[AGENT] >>> Node 1: ChatQwen
# 儿歌：小星星的夜游
## 第一段：星空奇遇
<OUTLINE>
扩写要求：
- 描述小星星离开天空的家，去地面探险的情景。
- 引入小动物（如小兔、小猫）的惊喜相遇，增添温馨气氛。
- 字数限制：20字
</OUTLINE>

## 第二段：月下共舞
<OUTLINE>
扩写要求：
- 继续小星星的旅程，描述它与地面上的朋友在月光下跳舞的欢乐场景。
- 表达友情与快乐的主题，简述自然界的宁静美好。
- 字数限制：20字
</OUTLINE>

[AGENT] >>> Node 2: FromOutline
[AGENT] 执行扩写任务 <7138-776-002>：
扩写要求：
- 描述小星星离开天空的家，去地面探险的情景。
- 引入小动物（如小兔、小猫）的惊喜相遇，增添温馨气氛。
- 字数限制：20字
小星星跃出银河边，轻旋降落在林间。小兔小猫齐惊叹，月下新友共翩跹。

[AGENT] 执行扩写任务 <7138-085-005>：
扩写要求：
- 继续小星星的旅程，描述它与地面上的朋友在月光下跳舞的欢乐场景。
- 表达友情与快乐的主题，简述自然界的宁静美好。
- 字数限制：20字
小星星落树梢，与林间小动物，月光轻抚下，共舞谱欢笑，宁静夜更妙。



'小星星落树梢，与林间小动物，月光轻抚下，共舞谱欢笑，宁静夜更妙。'

In [10]:
writer.runnables[0].memory

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 请按照需要输出一份写作提纲\n4. 提纲的构成应当由多个阔系额要求构成\n5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等\n5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹\n6. 你只能输出提纲，不要输出具体的扩写内容\n\n**你的任务是:**\n写一首两段儿歌，每段20个字即可，策划简单一点\n\n\n**输出例子1**\n```\n# 第一章 标题1\n## 一、XXXX\n<OUTLINE>\n扩写摘要:\n对扩写内容做摘要总结\n\n扩写要求：\n- xxx\n- xxx\n</OUTLINE>\n\n## 二、XXXX\n<OUTLINE>\n</OUTLINE>\n\n## （更多扩写提纲）\n```\n\n'},
 {'role': 'user', 'content': '请开始编写提纲'},
 {'role': 'assistant',
  'content': '# 儿歌：小星星的夜游\n## 第一段：星空奇遇\n<OUTLINE>\n扩写要求：\n- 描述小星星离开天空的家，去地面探险的情景。\n- 引入小动物（如小兔、小猫）的惊喜相遇，增添温馨气氛。\n- 字数限制：20字\n</OUTLINE>\n\n## 第二段：月下共舞\n<OUTLINE>\n扩写要求：\n- 继续小星星的旅程，描述它与地面上的朋友在月光下跳舞的欢乐场景。\n- 表达友情与快乐的主题，简述自然界的宁静美好。\n- 字数限制：20字\n</OUTLINE>'}]

In [11]:
print(writer.runnables[1].output)

# 儿歌：小星星的夜游

## 第一段：星空奇遇

小星星跃出银河边，轻旋降落在林间。小兔小猫齐惊叹，月下新友共翩跹。


## 第二段：月下共舞

小星星落树梢，与林间小动物，月光轻抚下，共舞谱欢笑，宁静夜更妙。




In [6]:
writer.runnables[1].runnables

{'6983-282-002': <illufly.agent.llm.dashscope.ChatQwen at 0x10cc2b7f0>,
 '6983-094-005': <illufly.agent.llm.dashscope.ChatQwen at 0x11e4c4d90>}

In [7]:
writer.runnables[1].runnables['6983-282-002'].memory

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 你必须根据已有提纲扩写，不要修改提纲中对扩写的要求和限定，不要额外发挥\n\n**已有文字草稿如下:**\n```markdown\n# 儿歌：快乐的小星星\n\n## 第一段\n\n&lt;&lt;&lt;YOUR_TEXT&gt;&gt;&gt;\n\n\n## 第二段\n\n...\n\n```\n\n**请你按照如下扩写任务要求生成一段文字，使其适合替换上面文字草稿中`<<<YOUR_TEXT>>>`所在位置:**\n```markdown\n扩写摘要:\n描绘夜空中闪烁的小星星，营造宁静而美好的夜晚景象。\n\n扩写要求：\n- 描述小星星在夜空中的状态（预估字数：10）\n- 表达小星星的快乐情绪（预估字数：10）\n```\n'},
 {'role': 'user', 'content': '请开始扩写'},
 {'role': 'assistant',
  'content': '# 儿歌：快乐的小星星\n\n## 第一段\n\n夜空如画卷展开，小星星眨眼闪亮。  \n它们轻盈跳跃，在银河里欢畅游荡。\n\n## 第二段\n\n...'}]

In [16]:
x = '```\n<OUTLINE>markdown\n清晨鸟儿窗前鸣，欢乐歌声醒梦魂。\n孩童蹦跳离床畔，满怀期待迎晨光。\n</OUTLINE>\n```'

from illufly.utils import extract_text
extract_text(extract_text(x, '```', '```'), "<OUTLINE>", "</OUTLINE>")

'清晨鸟儿窗前鸣，欢乐歌声醒梦魂。\n孩童蹦跳离床畔，满怀期待迎晨光。'

In [5]:
writer.runnables[1].runnables['5634-512-004'].memory

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 你必须根据已有提纲扩写，不要修改提纲中对扩写的要求和限定，不要额外发挥\n\n**已有文字草稿如下:**\n```markdown\n```\n# 儿歌：欢乐小天地\n\n## 第一段：动物乐园\n```\n\n...\n\n## 第二段：星空幻想\n\n&lt;&lt;&lt;YOUR_TEXT&gt;&gt;&gt;\n\n\n```\n```\n\n\n```\n\n**请你按照如下扩写任务要求生成一段文字，使其适合替换上面文字草稿中`<<<YOUR_TEXT>>>`所在位置:**\n请开始扩写\n'},
 {'role': 'assistant',
  'content': '```markdown\n夜晚来临，月亮船摇，\n星星点点，眨着眼笑。\n小熊星座，讲述古老，\n银河里藏，神秘的宝。\n\n小小梦想，飞出窗角，\n与流星赛跑，穿过云涛。\n宇宙的秘密，轻轻揭开，\n在梦的飞船，勇敢去探险。\n\n## 第三段：晨光冒险\n```'}]

In [19]:
print(writer.output)

# 儿歌：小星星的夜游


## 第一段：星星醒来

夜幕轻垂，万籁俱寂，星空渐渐亮起眼眸。小星星闪耀登场，好奇地眨，探秘夜的温柔。


## 第二段：月亮朋友

小星星遇见了月亮姐姐，手拉手舞动在夜空，欢笑声响彻云霄。




In [20]:
writer.runnables

(<illufly.agent.template.Template at 0x11ee69b10>,
 <illufly.agent.team.from_outline.FromOutline at 0x11ee3ae00>)